**Proyek Mini: Analisis Perilaku Pengguna**

**Konteks: Bayangkan Anda adalah seorang Data Analyst di sebuah perusahaan aplikasi mobile. Anda diberi dataset (Anda bisa membuat data tiruan sendiri) yang berisi informasi pengguna: user_id, age_category ('Di Bawah 25', 'Di Atas 25'), dan action_taken ('Membeli Produk', 'Tidak Membeli').**

----

In [14]:
import pandas as pd
import numpy as np 

user_df = pd.read_csv('analisis_pengguna.csv')
user_df.head()

,user_id,age_category,action_taken
0,1,Di Atas 25,Membeli Produk
1,2,Di Bawah 25,Tidak Membeli
2,3,Di Atas 25,Tidak Membeli
3,4,Di Atas 25,Tidak Membeli
4,5,Di Bawah 25,Membeli Produk


**TASK**

1. Uji Proporsi 1 Sampel:
- Hipotesis: Tim produk mengklaim bahwa 40% dari semua pengguna aplikasi berusia 'Di Bawah 25'. Ujilah klaim ini.
- Langkah: Hitung proporsi sampel ($\hat{p}$), tentukan proporsi hipotesis ($p_0=0.4$), dan hitung z-score serta p-value seperti pada materi.

- 

In [15]:
# One-Sample Proportion merupakan Test yang bertujuan untuk menguji klaim mengenai proporsi populasi


# proporsi sampel p_hat -> it's like dugaan hasil pengamatan sementara
p_hat = (user_df['age_category'] == 'Di Bawah 25').mean()
print('proporsi pengguna', p_hat)

# proporsi hipotesis (p_0)
p_0 = 0.4

# Hitung ukuran sampel (n)
n = len(user_df)
print('ukuran sample', n)

# hitung nila z-score
numerator = p_hat - p_0 
denominator = np.sqrt(p_0 * (1 - p_0) / n) 
z_score = numerator / denominator 
print('z-score:', z_score)
 
from scipy.stats import norm
# hitung p-value
p_value = 2 * (1 - norm.cdf(abs(z_score)))  # dua sisi
print("p-value:", p_value)

# taking decision
alpha = 0.05

if p_value < alpha:
    print("Tolak H0: proporsi berbeda signifikan dari", p_0)
else:
    print("Gagal menolak H0: tidak ada bukti signifikan bahwa proporsi berbeda dari", p_0)

proporsi pengguna 0.5
ukuran sample 30
z-score: 1.1180339887498947
p-value: 0.26355247728297293
Gagal menolak H0: tidak ada bukti signifikan bahwa proporsi berbeda dari 0.4


---
2. Uji Proporsi 2 Sampel:
- Hipotesis: Selidiki apakah ada perbedaan signifikan dalam proporsi pembelian produk antara pengguna 'Di Bawah 25' dan 'Di Atas 25'.
-  Langkah: Buat hipotesis nol ($H_0: p_{<25} - p_{\ge25} = 0$). Gunakan fungsi proportions_ztest dari statsmodels untuk menghitung z-score dan p-value.

    - Hipotesis nol (H₀): Tidak ada perbedaan proporsi pembelian antara dua kelompok umur H0​:p1​=p2
    - Hipotesis alternatif (H₁): Ada perbedaan proporsi pembelian antara dua kelompok umur H1​:p1​ tidak sama dengan p2​ 



In [16]:
# Hitung jumlah masing-masing kategori
count_table = user_df.groupby(["age_category", "action_taken"]).size().unstack()
print(count_table)
print('=' * 50)

# Hitung proporsi pembeli per kelompok usia
p_hats = user_df.groupby("age_category")['action_taken'].value_counts(normalize=True)
print(p_hats)
print('=' * 50)

# Ambil proporsi 'Membeli Produk'
p_hat_atas25 = p_hats[("Di Atas 25", "Membeli Produk")]
p_hat_bawah25 = p_hats[("Di Bawah 25", "Membeli Produk")]
print('p_hat_upper 25:', p_hat_atas25)
print('p_hat under 25:', p_hat_bawah25)
print('=' * 50)

# Hitung jumlah observasi masing-masing grup
n = user_df.groupby("age_category")['action_taken'].count()
n_atas25 = n["Di Atas 25"]
n_bawah25 = n["Di Bawah 25"]


# Pooled proportion (gabungan)
p_hat = (n_atas25*p_hat_atas25 + n_bawah25*p_hat_bawah25) / (n_atas25 + n_bawah25)

# Standard error
std_error = np.sqrt(p_hat * (1 - p_hat) * (1/n_atas25 + 1/n_bawah25))

# Z-score
z_score = (p_hat_atas25 - p_hat_bawah25) / std_error
print("Z-score:", z_score)

from scipy.stats import norm

# Dua arah
p_value = 2 * (1 - norm.cdf(abs(z_score)))
print("p-value:", p_value)


print('=' * 50) 
print('+' * 50)
#####################
from statsmodels.stats.proportion import proportions_ztest

# hitung jumlah "Membeli Produk" di masing-masing kategori usia
count = user_df.groupby('age_category')['action_taken'].apply(lambda x: (x == 'Membeli Produk').sum())
nobs = user_df['age_category'].value_counts()
# print("Jumlah pembeli (count):\n", count)
# print("Jumlah total (nobs):\n", nobs)

# konversi ke urutan list agar sejalan
count = count.values
nobs = nobs.values


# uji dua sisi
z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

print("Z-stat:", z_stat)
print("P-value:", p_value)

print('=' * 50)
# decision
alpha = 0.05
if p_value < alpha:
    print("Tolak H0: proporsi berbeda signifikan antara kedua kelompok usia")
else:
    print("Gagal menolak H0: tidak ada bukti signifikan bahwa proporsi berbeda antara kedua kelompok usia") 
    


action_taken  Membeli Produk  Tidak Membeli
age_category                               
Di Atas 25                 7              8
Di Bawah 25                5             10
age_category  action_taken  
Di Atas 25    Tidak Membeli     0.533333
              Membeli Produk    0.466667
Di Bawah 25   Tidak Membeli     0.666667
              Membeli Produk    0.333333
Name: proportion, dtype: float64
p_hat_upper 25: 0.4666666666666667
p_hat under 25: 0.3333333333333333
Z-score: 0.74535599249993
p-value: 0.4560565402502559
++++++++++++++++++++++++++++++++++++++++++++++++++
Z-stat: 0.74535599249993
P-value: 0.4560565402502559
Gagal menolak H0: tidak ada bukti signifikan bahwa proporsi berbeda antara kedua kelompok usia


---
3. Uji Chi-Square Independensi (Opsional, jika ada data tambahan):

- Hipotesis: Jika Anda memiliki data tambahan seperti satisfaction_level ('Puas', 'Netral', 'Tidak Puas'), ujilah apakah ada hubungan antara age_category dan satisfaction_level.
- Langkah: Gunakan fungsi pingouin.chi2_independence untuk melakukan pengujian.

In [17]:
# Tambahkan kolom baru satisfaction_level secara acak

np.random.seed(42)  # supaya hasilnya konsisten kalau dijalankan ulang
user_df['satisfaction_level'] = np.random.choice(['Puas', 'Netral', 'Tidak Puas'], size=len(user_df))

user_df.head()

,user_id,age_category,action_taken,satisfaction_level
0,1,Di Atas 25,Membeli Produk,Tidak Puas
1,2,Di Bawah 25,Tidak Membeli,Puas
2,3,Di Atas 25,Tidak Membeli,Tidak Puas
3,4,Di Atas 25,Tidak Membeli,Tidak Puas
4,5,Di Bawah 25,Membeli Produk,Puas


- H0 : tidak ada hubungan signifikan antara categori usia dan level kepuasan
- H1 : ada hubungan signifikan antara categori usia dan level kepuasan

In [18]:
# Uji Chi-Square selalu butuh tabulasi silang (crosstab)

contingency_table = pd.crosstab(user_df["age_category"], user_df["satisfaction_level"])
print(contingency_table)

print('=' * 50)
print('=' * 50)
# uji Chi-Square dengan pingouin
import pingouin as pg

expected, observed, stats = pg.chi2_independence(data=user_df, x='age_category', y='satisfaction_level')


# print(expected)  # tabel expected frequencies
# print(observed)  # tabel observed frequencies
print(stats)     # statistik uji chi-square


satisfaction_level  Netral  Puas  Tidak Puas
age_category                                
Di Atas 25               4     4           7
Di Bawah 25              4     6           5
                 test    lambda      chi2  dof      pval    cramer     power
0             pearson  1.000000  0.733333  2.0  0.693041  0.156347  0.109335
1        cressie-read  0.666667  0.734278  2.0  0.692713  0.156448  0.109416
2      log-likelihood  0.000000  0.737604  2.0  0.691562  0.156802  0.109703
3       freeman-tukey -0.500000  0.741378  2.0  0.690259  0.157202  0.110028
4  mod-log-likelihood -1.000000  0.746270  2.0  0.688572  0.157720  0.110450
5              neyman -2.000000  0.759524  2.0  0.684024  0.159115  0.111595


c:\Users\USER\AppData\Local\Programs\Python\Python314\Lib\site-packages\pingouin\contingency.py:151: UserWarning: Low count on observed frequencies.
  warnings.warn(f"Low count on {name} frequencies.")
c:\Users\USER\AppData\Local\Programs\Python\Python314\Lib\site-packages\pingouin\contingency.py:151: UserWarning: Low count on expected frequencies.
  warnings.warn(f"Low count on {name} frequencies.")


- Berdasarkan hasil uji Chi-Square (p-value = 0.693 > 0.05), tidak terdapat hubungan yang signifikan antara kategori usia dan tingkat kepuasan pengguna.
